In [1]:
from data_processing import generate_csf_image
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm


In [2]:
def calculate_T_in_pix(spatial_frequency,distance_in_meter,dpi):
    mm_per_degree = 1000 * distance_in_meter * np.tan(2 * np.pi / 360)
    T_in_mm = (1 / spatial_frequency) * mm_per_degree
    T_in_pixel = T_in_mm * dpi / 25.4
    return int(T_in_pixel)
def calculate_size_in_pix(size_in_mm,dpi):
    size_in_pixel = size_in_mm * dpi / 25.4
    return int(size_in_pixel)

In [3]:
dpi = 1200
size_in_mm = int(270/4)
distance_in_meter=5
size =calculate_size_in_pix(size_in_mm, dpi)
T_list_in_cycle_per_deg = [3,6,12,18,24] 
# logCS=np.array([
#     # [0.85,0.95,1.11,1.26,1.40,1.56,1.70,1.85,2.00],
#     [1.00,1.18,1.30,1.46,1.60,1.76,1.90,2.05,2.20],
#     [1.08,1.20,1.36,1.52,1.65,1.81,1.95,2.11,2.26],
#     [0.90,1.04,1.18,1.34,1.48,1.63,1.78,1.93,2.08],
#     [0.60,0.78,0.90,1.08,1.23,1.36,1.52,1.66,1.81]
# ])
logCS=np.linspace(1.0,2.0,12)
contract_list = np.tile(10**(-logCS), (len(T_list_in_cycle_per_deg),1))
T_list_in_pix = [calculate_T_in_pix(T, distance_in_meter, dpi) for T in T_list_in_cycle_per_deg]
T_list = np.tile(T_list_in_pix, (logCS.shape[0], 1)).T
contract_list=contract_list.flatten()
T_list=T_list.flatten()
text_list = '●'*len(T_list)
angle_list = np.random.randint(-90, 90, len(text_list)) // 45 * 45



In [14]:
tstring = ""
for cs in 10**(-logCS):
    tstring+=f"{cs:.3f} "
print(tstring)

0.100 0.081 0.066 0.053 0.043 0.035 0.028 0.023 0.019 0.015 0.012 0.010 


In [4]:

avg_value = 127
blur_core=size//20*2+1
blur_radius = blur_core //2

# # for i,text in enumerate(text_list):
#     T = T_list[i]
#     angle = angle_list[i]
#     contrast = contrast_list[i]
i = 0 
for text, T, contrast, angle in zip(text_list, T_list, contract_list, angle_list):

    csf_image = generate_csf_image(size, T, contrast, angle, avg_value, text, blur_core, blur_radius)
    # print(csf_image.shape)
    fig, ax = plt.subplots(figsize=(csf_image.shape[1] / dpi, csf_image.shape[0] / dpi), dpi=dpi)
    ax.imshow(csf_image, cmap='gray', vmin=0, vmax=255)
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(f'image_{i}.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    i += 1


size: 3165, T: 1374, contrast: 0.10, angle: 45
size: 3165, T: 1374, contrast: 0.08, angle: -90
size: 3165, T: 1374, contrast: 0.07, angle: -90
size: 3165, T: 1374, contrast: 0.05, angle: 0
size: 3165, T: 1374, contrast: 0.04, angle: -90
size: 3165, T: 1374, contrast: 0.04, angle: 45
size: 3165, T: 1374, contrast: 0.03, angle: 45
size: 3165, T: 1374, contrast: 0.02, angle: 45
size: 3165, T: 1374, contrast: 0.02, angle: -90
size: 3165, T: 1374, contrast: 0.02, angle: -90
size: 3165, T: 1374, contrast: 0.01, angle: -45
size: 3165, T: 1374, contrast: 0.01, angle: 0
size: 3165, T: 687, contrast: 0.10, angle: 45
size: 3165, T: 687, contrast: 0.08, angle: 0
size: 3165, T: 687, contrast: 0.07, angle: -90
size: 3165, T: 687, contrast: 0.05, angle: -90
size: 3165, T: 687, contrast: 0.04, angle: -45
size: 3165, T: 687, contrast: 0.04, angle: -90
size: 3165, T: 687, contrast: 0.03, angle: 0
size: 3165, T: 687, contrast: 0.02, angle: 0
size: 3165, T: 687, contrast: 0.02, angle: 45
size: 3165, T: 68

In [5]:
# 每页放置 12 张图像，3x4 均匀摆放
images_per_page = 12
rows = 4
cols = 3

# 创建 PDF
# 将 A4 尺寸从点转换为像素
width, height = [dim * dpi / 72 for dim in [8.3*72, 11.7*72]]

# 创建 PDF 并设置页面大小
c = canvas.Canvas("output.pdf", pagesize=(width, height))

image_width = width / cols
image_height = height / rows


for i in range(0, len(T_list), images_per_page):
    for j in range(images_per_page):
        if i + j >= len(T_list):
            break
        img_path = f'image_{i + j}.png'
        x = (j % cols) * image_width
        y = height - ((j // cols) + 1) * image_height
        c.drawImage(img_path, x, y, width=size, height=size)
    c.showPage()

c.save()